In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amazon.csv to Amazon.csv
Saving Amgen.csv to Amgen.csv
Saving Apple.csv to Apple.csv
Saving Costco.csv to Costco.csv
Saving Google.csv to Google.csv
Saving Meta.csv to Meta.csv
Saving Microsoft.csv to Microsoft.csv
Saving Netflix.csv to Netflix.csv
Saving Starbucks.csv to Starbucks.csv
Saving Tesla.csv to Tesla.csv


In [ ]:
import pandas as pd
import numpy as np

# Ticker mappings (rename as needed)
file_map = {
    "Amazon.csv": "AMZN",
    "Amgen.csv": "AMGN",
    "Apple.csv": "AAPL",
    "Costco.csv": "COST",
    "Google.csv": "GOOGL",
    "Meta.csv": "META",
    "Microsoft.csv": "MSFT",
    "Netflix.csv": "NFLX",
    "Starbucks.csv": "SBUX",
    "Tesla.csv": "TSLA"
}

In [ ]:
price_data = []

for filename, ticker in file_map.items():
    df = pd.read_csv(filename, parse_dates=['Date'])
    df = df[['Date', 'Price']].rename(columns={'Price': ticker})
    price_data.append(df)

# Merge all on 'Date'
merged = price_data[0]
for df in price_data[1:]:
    merged = pd.merge(merged, df, on='Date', how='inner')  # inner join = keep only common dates

merged = merged.sort_values('Date').set_index('Date')

In [ ]:
missing_counts = merged.isnull().sum()
print("Missing values per column:\n", missing_counts)

# Drop rows with any missing values (safer for modeling)
merged_clean = merged.dropna()

Missing values per column:
 AMZN     0
AMGN     0
AAPL     0
COST     0
GOOGL    0
META     0
MSFT     0
NFLX     0
SBUX     0
TSLA     0
dtype: int64


In [ ]:
log_returns = np.log(merged_clean / merged_clean.shift(1)).dropna()
log_returns.head()

,AMZN,AMGN,AAPL,COST,GOOGL,META,MSFT,NFLX,SBUX,TSLA
Date,,,,,,,,,,
2015-01-05,-0.020957,-0.011954,-0.028579,-0.011446,-0.019448,-0.016192,-0.009238,-0.052297,-0.019341,-0.042619
2015-01-06,-0.023436,-0.032748,0.000376,0.013123,-0.024953,-0.013565,-0.014786,-0.017269,-0.008044,0.005694
2015-01-07,0.010782,0.034329,0.013829,0.017257,-0.002767,0.000000,0.012625,0.005148,0.024188,-0.002131
2015-01-08,0.006680,-0.003609,0.037520,0.008554,0.003557,0.026181,0.028994,0.022007,0.015887,-0.001423
2015-01-09,-0.011383,-0.012381,0.001072,-0.015522,-0.012304,-0.005516,-0.008441,-0.015609,-0.033032,-0.018692


In [ ]:
from google.colab import files

# Save to CSV if not already done
log_returns.to_csv("nasdaq10_log_returns.csv")

# Download to your computer
files.download("nasdaq10_log_returns.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define ticker symbols
tickers = {
    "SPY": "SPY",         # S&P 500 ETF
    "QQQ": "QQQ",         # Nasdaq-100 ETF
    "VIX": "^VIX"         # Volatility Index
}

# Download adjusted close prices
data = yf.download(list(tickers.values()), start="2015-01-01", end="2025-01-01")['Close']

# Compute log returns for SPY and QQQ
spy_logret = np.log(data['SPY'] / data['SPY'].shift(1)).rename('SPY_ret')
qqq_logret = np.log(data['QQQ'] / data['QQQ'].shift(1)).rename('QQQ_ret')

# Use raw VIX values (level, not return)
vix_raw = data['^VIX'].rename('VIX')

# Combine all into one DataFrame
exogenous_features = pd.concat([spy_logret, qqq_logret, vix_raw], axis=1).dropna()

# Save or preview
print(exogenous_features.head())
exogenous_features.to_csv("exogenous_features.csv")

/tmp/ipython-input-5-1800247497.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(list(tickers.values()), start="2015-01-01", end="2025-01-01")['Close']
[*********************100%***********************]  3 of 3 completed

             SPY_ret   QQQ_ret        VIX
Date                                     
2015-01-05 -0.018224 -0.014778  19.920000
2015-01-06 -0.009464 -0.013499  21.120001
2015-01-07  0.012384  0.012809  19.309999
2015-01-08  0.017589  0.018958  17.010000
2015-01-09 -0.008046 -0.006604  17.549999


In [ ]:
from google.colab import files
files.download("exogenous_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amazon.csv to Amazon.csv
Saving Amgen.csv to Amgen.csv
Saving Apple.csv to Apple.csv
Saving Costco.csv to Costco.csv
Saving Google.csv to Google.csv
Saving Meta.csv to Meta.csv
Saving Microsoft.csv to Microsoft.csv
Saving Netflix.csv to Netflix.csv
Saving Starbucks.csv to Starbucks.csv
Saving Tesla.csv to Tesla.csv


In [ ]:
import pandas as pd
import numpy as np
import os

# File map: Stock name to file name
file_map = {
    "Amazon": "Amazon.csv",
    "Amgen": "Amgen.csv",
    "Apple": "Apple.csv",
    "Costco": "Costco.csv",
    "Google": "Google.csv",
    "Meta": "Meta.csv",
    "Microsoft": "Microsoft.csv",
    "Netflix": "Netflix.csv",
    "Starbucks": "Starbucks.csv",
    "Tesla": "Tesla.csv"
}

# Function to compute features and rename columns
def compute_features(df, stock_name):
    price = df["Price"]
    log_returns = np.log(price / price.shift(1))
    # Lagged Returns
    for lag in range(1, 6):
        df[f"{stock_name}_lag_return_{lag}d"] = log_returns.shift(lag)

    # EMA
    df[f"{stock_name}_ema_12"] = price.ewm(span=12, adjust=False).mean()
    df[f"{stock_name}_ema_26"] = price.ewm(span=26, adjust=False).mean()

    # MACD
    df[f"{stock_name}_macd_line"] = df[f"{stock_name}_ema_12"] - df[f"{stock_name}_ema_26"]
    df[f"{stock_name}_macd_signal"] = df[f"{stock_name}_macd_line"].ewm(span=9, adjust=False).mean()

    # RSI
    delta = price.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df[f"{stock_name}_rsi_14"] = 100 - (100 / (1 + rs))

    # Keep only new features + Date
    df = df[[col for col in df.columns if col.startswith(stock_name)]]
    return df

# Initialize the final merged DataFrame
merged_df = None

# Loop through each stock
for stock_name, filename in file_map.items():
    df = pd.read_csv(filename, parse_dates=["Date"])
    df.set_index("Date", inplace=True)
    df = df.sort_index()

    # Compute features
    features_df = compute_features(df, stock_name)

    # Merge
    if merged_df is None:
        merged_df = features_df
    else:
        merged_df = merged_df.join(features_df, how="outer")

# Drop rows with any NaN (optional)
merged_df.dropna(inplace=True)

# Save to CSV
merged_df.to_csv("all_stock_features.csv")
print("✅ All stock features saved to all_stock_features.csv")

✅ All stock features saved to all_stock_features.csv


In [ ]:
from google.colab import files
files.download("all_stock_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving all_stock_features.csv to all_stock_features (3).csv
Saving exogenous_features.csv to exogenous_features (1).csv
Saving nasdaq10_log_returns.csv to nasdaq10_log_returns (1).csv


In [ ]:
import pandas as pd

# Load all three files
features_df = pd.read_csv("all_stock_features.csv", parse_dates=["Date"])
exog_df = pd.read_csv("exogenous_features.csv", parse_dates=["Date"])
returns_df = pd.read_csv("nasdaq10_log_returns.csv", parse_dates=["Date"])

# Set index to Date for alignment
features_df.set_index("Date", inplace=True)
exog_df.set_index("Date", inplace=True)
returns_df.set_index("Date", inplace=True)

# Find common date range across all 3 datasets
common_dates = features_df.index.intersection(exog_df.index).intersection(returns_df.index)

# Filter datasets to only include common dates
features_df = features_df.loc[common_dates]
exog_df = exog_df.loc[common_dates]
returns_df = returns_df.loc[common_dates]

# Reset index for convenience (optional)
features_df.reset_index(inplace=True)
exog_df.reset_index(inplace=True)
returns_df.reset_index(inplace=True)

# Summary print
print("✅ Data aligned across all 3 datasets")
print(f"📅 Common date range: {common_dates.min().date()} to {common_dates.max().date()}")
print(f"🔢 Total aligned rows: {len(common_dates)}")

# Save aligned data to new CSVs
features_df.to_csv("aligned_all_stock_features.csv", index=False)
exog_df.to_csv("aligned_exogenous_features.csv", index=False)
returns_df.to_csv("aligned_nasdaq10_log_returns.csv", index=False)

✅ Data aligned across all 3 datasets
📅 Common date range: 2015-01-23 to 2024-12-31
🔢 Total aligned rows: 2502


In [ ]:
from google.colab import files
files.download("aligned_all_stock_features.csv")
files.download("aligned_exogenous_features.csv")
files.download("aligned_nasdaq10_log_returns.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>